In [ ]:
#Assignment 5
import pandas as pd

In [ ]:
import os
path_files_folder1 = "/content/drive/MyDrive/EETBS 2023-2024/RLF Tables "
name_windows_files = "windows .csv"
path_windows_file = os.path.join(path_files_folder1,name_windows_files)
windows_DF = pd.read_csv(path_windows_file, sep=";", index_col =0, header = 0)
windows_DF

,Direction,width,Height,Area,Window_ID,Frame_type,Frame_material,U,SHGC,HF,...,IAC_cl,IAC,Tx,Ed,ED,PXI,FFs,C_value,CF,Qcooling
Name,,,,,,,,,,,,,,,,,,,,,
east,E,8,1.8,0,5c,Fixed,Wood,0,0,0,...,0,0,1.00,0,0,0,0,0,0,0
west,W,8,1.8,0,5c,Fixed,Wood,0,0,0,...,0,0,1.00,0,0,0,0,0,0,0
south-Fixed,S,2,1.8,0,5c,Fixed,Wood,0,0,0,...,0,0,1.00,0,0,0,0,0,0,0
south-Operable,S,2,1.8,0,5c,Operable,Wood,0,0,0,...,0,0,0.64,0,0,0,0,0,0,0


In [ ]:
windows_DF.columns #to show all columns in table

Index(['Direction', 'width', 'Height', 'Area', 'Window_ID', 'Frame_type',
       'Frame_material', 'U', 'SHGC', 'HF', 'Q heating', 'SLF', 'Doh', 'Xoh',
       'Fshd', 'IntShading_ID', 'IntShading_closeness', 'IAC_cl', 'IAC', 'Tx',
       'Ed', 'ED', 'PXI', 'FFs', 'C_value', 'CF', 'Qcooling'],
      dtype='object')

In [ ]:
windows_DF.loc[:,["Ed", "ED", "SLF", "Fshd", "PXI", ]]

,Ed,ED,SLF,Fshd,PXI
Name,,,,,
east,0,0,0,0,0
west,0,0,0,0,0
south-Fixed,0,0,0,0,0
south-Operable,0,0,0,0,0


In [ ]:
# E_Direct
def BeamIrrad_finder(row):
  direction=row["Direction"]
  path_files_folder1 = "/content/drive/MyDrive/EETBS 2023-2024/RLF Tables "
  name_file_BeamIrrad = "BeamIrradiance.csv"
  path_file_BeamIrrad = os.path.join(path_files_folder1,name_file_BeamIrrad)
  BeamIrrad_DF = pd.read_csv(path_file_BeamIrrad, sep=";", index_col = 0, header = 0)
  value_BeamIrrad = BeamIrrad_DF.loc[direction,"45"]
  return (value_BeamIrrad)
# E diffuse
def DiffIrrad_finder(row):
  direction=row["Direction"]
  path_files_folder2 = "/content/drive/MyDrive"
  name_file_DiffIrrad = "DiffuseIrradiance.csv"
  path_file_DiffIrrad = os.path.join(path_files_folder2,name_file_DiffIrrad)
  DiffIrrad_DF = pd.read_csv(path_file_DiffIrrad, sep=";", index_col = 0, header = 0)
  value_DiffIrrad = DiffIrrad_DF.loc[direction,"45"]
  return (value_DiffIrrad)
#SLF
def SLF_finder(row):
  direction=row["Direction"]
  name_file_SLF = "SLF.xlsx"
  path_file_SLF = os.path.join(path_files_folder1,name_file_SLF)
  SLF_DF = pd.read_excel(path_file_SLF, index_col = 0, header = 0)
  value_SLF = SLF_DF.loc[direction, 45]
  return (value_SLF)
#F_shd
def F_shd_finder(row):
  SLF=row["SLF"]
  Xoh=row["Xoh"]
  Doh=row["Doh"]
  if row["Height"]==0:
    value_F_shd=0
  else:
    h=row["Height"]
    value_F_shd=min(1,max(0,(SLF*Doh-Xoh)/h))
    value_F_shd=round(value_F_shd,3)
  return (value_F_shd)
# PXI
def PXI_finder(row):
  Tx=row["Tx"]
  Ed=row["Ed"]
  ED=row["ED"]
  Fshd=row["Fshd"]
  if row["Height"]==0:
    value_PXI=0
  else:
    value_PXI=Tx*(Ed+(1-Fshd)*ED)
    value_PXI=round(value_PXI,3)
  return (value_PXI)

In [ ]:
windows_DF.apply(PXI_finder,axis=1)
windows_DF.loc[:,"PXI"] = windows_DF.apply(PXI_finder,axis=1)
windows_DF.apply(F_shd_finder,axis=1)
windows_DF.loc[:,"Fshd"] = windows_DF.apply(F_shd_finder,axis=1)
windows_DF.apply(SLF_finder,axis=1)
windows_DF.loc[:,"SLF"] = windows_DF.apply(SLF_finder,axis=1)
windows_DF.apply(DiffIrrad_finder,axis=1)
windows_DF.loc[:,"Ed"] = windows_DF.apply(DiffIrrad_finder,axis=1)
windows_DF.apply(BeamIrrad_finder,axis=1)
windows_DF.loc[:,"ED"] = windows_DF.apply(BeamIrrad_finder,axis=1)
windows_DF.loc[:,["Ed", "ED", "SLF", "Fshd", "PXI"]]

,Ed,ED,SLF,Fshd,PXI
Name,,,,,
east,188,559,1.0,0.333,560.667
west,188,559,1.0,0.333,560.667
south-Fixed,209,348,2.7,1.000,209.000
south-Operable,209,348,2.7,1.000,133.760


In [ ]:
path_modifiedWindowsFile="/content/drive/MyDrive/EETBS 2023-2024/RLF Tables Students Copy/ElizavetaMukhlina/Assignment5_ElizavetaMukhlina_modified.xlsx"
windows_DF.to_excel(path_modifiedWindowsFile)